In [1]:
print("hello")

hello


In [51]:
import os
import json

from Work.Helper import *

all_files= list_all_tx() 
    
   
   


[{'version': 2, 'locktime': 0, 'vin': [{'txid': '64ca1941edef34b690dd6672c7d395c60882067f7f3fc396e64d88e39c1da5b4', 'vout': 0, 'prevout': {'scriptpubkey': '0014d5bfb7a6d05d44c1e14443919b30d284c0c0a10a', 'scriptpubkey_asm': 'OP_0 OP_PUSHBYTES_20 d5bfb7a6d05d44c1e14443919b30d284c0c0a10a', 'scriptpubkey_type': 'v0_p2wpkh', 'scriptpubkey_address': 'bc1q6klm0fkst4zvrc2ygwgekvxjsnqvpgg2jjfurm', 'value': 10740}, 'scriptsig': '', 'scriptsig_asm': '', 'witness': ['3044022100884219ecbb54a6ec4d09597ca6aca49692ded3c2ffb13d1858ca5b70e59fabb4021f2de73021471a01d8f03a71a923b662f00120d181d0f7fa8e06faa1bb750e8f01', '0271d4e7a84804c075017593271c370e8983f704f123d22aa747cd321268981cba'], 'is_coinbase': False, 'sequence': 4294967293}], 'vout': [{'scriptpubkey': 'a91450feb99697a4901d3fe082eca341204fb6711b9487', 'scriptpubkey_asm': 'OP_HASH160 OP_PUSHBYTES_20 50feb99697a4901d3fe082eca341204fb6711b94 OP_EQUAL', 'scriptpubkey_type': 'p2sh', 'scriptpubkey_address': '395H8VPYPtAoZWa2bx5SRyN2VojXrsb7j3', 'value': 

8131

In [70]:
scriptpubkey_types=set()

for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vin":     
          scriptpubkey_types.add(item["prevout"]["scriptpubkey_type"])
        else:  
           scriptpubkey_types.add(item["scriptpubkey_type"])
scriptpubkey_types  
          


{'op_return', 'p2pkh', 'p2sh', 'unknown', 'v0_p2wpkh', 'v0_p2wsh', 'v1_p2tr'}

In [52]:
# op_return -> 
op_return_in_input=set()
op_return_in_output=set()



for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vin":  
          if (item["prevout"]["scriptpubkey_type"])=='op_return':
             op_return_in_input.add(item["prevout"]["value"])
             
          scriptpubkey_types.add(item["prevout"]["scriptpubkey_type"])
        else:
            if (item["scriptpubkey_type"])=='op_return':
             op_return_in_output.add(item["value"])
               
print(op_return_in_input)
print(op_return_in_output)


set()
{0}


In [47]:
# is 0 value in vin -> 
count=0
for tx in all_files:
  for item in tx["vin"]:            
          if (item["prevout"]["value"])==0:
             count+=1
             
print(count) 



0


### No value 0 in input field

In [73]:
### First we will check that these keyword are present in both inputs and outputs field or not> 


unknown_keyword= {
   "vin":0,
   "vout":0
}

unique_commands_for_unknown=set()
trial=set()

for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vout" and  item["scriptpubkey_type"]=="unknown":     
          unknown_keyword["vout"]+=1
        if section == "vin" and  item["prevout"]["scriptpubkey_type"]=="unknown":  
            unknown_keyword["vin"]+=1

        #### In some places , "unknown" keyword is written inside the scriptpubkey_type 
        ### -> but it is actually Multisig as shown ->
        
        
        if section == "vout":
           if item["scriptpubkey_type"]=="unknown":
              unique_commands_for_unknown.add(item["scriptpubkey_asm"].split()[-1])
              trial.add(item["scriptpubkey"][-2:])
              

print("unknown keyword present : {}".format(unknown_keyword))   
print("unique_commands for the unknown type at the end {}".format(unique_commands_for_unknown))
print(trial)


unknown keyword present : {'vin': 0, 'vout': 81}
unique_commands for the unknown type at the end {'OP_CHECKMULTISIG'}
{'ae'}


In [74]:
for file in all_files:
    for output in file["vout"]:
        script_type = output["scriptpubkey_type"]
        if script_type == "unknown":
            output["scriptpubkey_type"] = "multisig"


In [76]:
import os
import json

folder_path = "./mempool"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for output in data["vout"]:
        script_type = output["scriptpubkey_type"]
        if script_type == "unknown":
            output["scriptpubkey_type"] = "bare-multisig"
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)
